### [Türkçe Film Yorumları Duygu Analizi (Başlangıç Seviyesi)

Bu proje, Kaggle’daki “Turkish Movie Sentiment Analysis Dataset” veri setini kullanarak Türkçe film yorumlarında duygu analizi yapmayı öğretir. Lojistik Regresyon modeli ile olumlu ve olumsuz duyguları sınıflandıracağız. Kod, NLP’ye yeni başlayanlar için sade ve anlaşılırdır.

### 1. Gerekli Kütüphaneleri Yükleme

Ne yapıyoruz?
Veriyi işlemek ve model oluşturmak için gerekli Python kütüphanelerini yüklüyoruz.

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 2. Veri Setini Yükleme

Ne yapıyoruz?
Kaggle’daki “Turkish Movie Sentiment Analysis Dataset” veri setini yüklüyoruz ve ilk 5 satırı görüntülüyoruz.

In [2]:
df = pd.read_csv('/kaggle/input/turkish-movie-sentiment-analysis-dataset/turkish_movie_sentiment_dataset.csv')
print(df.head())

                                             comment      film_name point
0  \n                      Jean Reno denince zate...  Sevginin Gücü   5,0
1  \n                      Ekşın falan izlemek is...  Sevginin Gücü   5,0
2  \n                      Bu yapım hakkında öyle...  Sevginin Gücü   5,0
3  \n                      finali yeter... (sting...  Sevginin Gücü   5,0
4  \n                      Jean Reno..\r\nbu adam...  Sevginin Gücü   5,0


### Sınıfları Azaltma

Ne yapıyoruz?
16 farklı puanı 3 sınıfa indiriyoruz: olumlu (4.0-5.0), nötr (3.0-3.9), olumsuz (0.5-2.5).

In [3]:
def simplify_labels(point):
    point = float(point.replace(',', '.'))  # Virgülü noktaya çevir
    if point >= 4.0:
        return 'olumlu'
    elif point >= 3.0:
        return 'nötr'
    else:
        return 'olumsuz'

df['label'] = df['point'].apply(simplify_labels)
print(df['label'].value_counts())

label
olumlu     43336
olumsuz    25397
nötr       14494
Name: count, dtype: int64


#### Not: Veri setinde genellikle comment (yorum) ve point (duygu etiketi, örneğin 1=olumlu, 0=olumsuz) sütunları bulunur. Sütun isimlerini kontrol etmek için print(df.columns) komutunu çalıştırın.

### 3. Metinleri Temizleme

Ne yapıyoruz?
Türkçe metinleri işlemek için şunları yapıyoruz:





Harfleri küçük harfe çeviriyoruz.



Noktalama işaretlerini ve özel karakterleri kaldırıyoruz.



Türkçe gereksiz kelimeleri (stopwords) çıkarıyoruz.

In [4]:
# Türkçe stopwords listesi ve eklemeler
stop_words = set(stopwords.words('turkish'))
stop_words.update(['ve', 'ile', 'de', 'da', 'bir', 'bu', 'şu', 'o'])

def clean_text(text):
    # Küçük harfe çevir
    text = text.lower()
    # Noktalama işaretlerini kaldır
    text = re.sub(r'[^\w\s]', '', text)
    # Stopwords'leri kaldır
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['comment'] = df['comment'].apply(clean_text)
print(df['comment'].head())

0    jean reno denince zaten leon filmi gelir akla ...
1    ekşın falan izlemek istiyorsanız bunu izlemeyi...
2    yapım hakkında öyle yazabilirim kitap olur yüz...
3    finali yeter sting shape of my heart bazıları ...
4    jean reno adam kusursuz oyunculugu müthiş film...
Name: comment, dtype: object


### 4. Metinleri Sayısal Verilere Çevirme

Ne yapıyoruz?
Metinleri makine öğrenmesi modelinin anlayacağı sayılara çevirmek için TF-IDF kullanıyoruz.

In [5]:
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(df['comment'])
y = df['label']

### 5. Veriyi Eğitim ve Test Setlerine Ayırma

Ne yapıyoruz?
Veriyi %80 eğitim, %20 test olarak ayırıyoruz.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 6. Modeli Eğitme
Ne yapıyoruz?
Lojistik Regresyon modelini class_weight='balanced' ile eğitiyoruz, böylece dengesiz sınıflar için daha iyi sonuç alırız.

In [7]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced')

### 7. Modeli Test Etme

Ne yapıyoruz?
Modelin doğruluğunu ve performansını test ediyoruz.

In [8]:
y_pred = model.predict(X_test)
print("Doğruluk Skoru:", accuracy_score(y_test, y_pred))
print("\nSınıflandırma Raporu:\n", classification_report(y_test, y_pred))

Doğruluk Skoru: 0.64700228283071

Sınıflandırma Raporu:
               precision    recall  f1-score   support

        nötr       0.37      0.51      0.43      2857
      olumlu       0.78      0.68      0.73      8626
     olumsuz       0.66      0.66      0.66      5163

    accuracy                           0.65     16646
   macro avg       0.60      0.62      0.61     16646
weighted avg       0.67      0.65      0.66     16646



### 8. Örnek Bir Yorumla Tahmin Yapma

Ne yapıyoruz?
Kendi yazdığımız bir film yorumunu modelle test ediyoruz.

In [9]:
test_yorum = "Bu film harika, çok keyif aldım!"
test_yorum_temiz = clean_text(test_yorum)
test_yorum_vec = vectorizer.transform([test_yorum_temiz])
tahmin = model.predict(test_yorum_vec)
print("Yorum:", test_yorum)
print("Tahmin:", tahmin[0])

Yorum: Bu film harika, çok keyif aldım!
Tahmin: olumlu


### Sonuç

Bu kod, Türkçe film yorumlarını olumlu, nötr veya olumsuz olarak sınıflandırır. Sınıf sayısını azaltarak ve dengesiz sınıfları dengeleyerek modeli iyileştirdik. Daha fazla geliştirmek istersen, başka veri temizleme yöntemleri veya modeller deneyebilirsin.